In [103]:
import numpy as np
import scipy.stats as scp

In [104]:
def AssetValueByMC(S0,T,sd,rf,e):
    ST = S0 * np.exp(T *(rf - (sd*sd)/2) + (sd*np.sqrt(T)*(e)))
    return ST

def Call(rf,T,ST,K):
    C = np.exp(-rf*T)*np.maximum(0,ST-K)
    return C

def CallBS(S0,K,T,sd,rf):
    d1 = 1/(sd*np.sqrt(T))*(np.log(S0/K) +T*(rf+1/2 * (sd**2)))
    d2 = d1 - sd*np.sqrt(T)
    C = S0 * scp.norm.cdf(d1)-K*scp.norm.cdf(d2)*np.exp(-rf*T)
    return C

def CallMonteCarlo(S0,K,T,sd,rf,iter):
    C = np.mean([Call(rf,T,AssetValueByMC(S0,T,sd,rf,i),K) for i in np.random.normal(0,1,iter)])
    return C

In [105]:
Prix_CallMC = CallMonteCarlo(S0=10,K=10,T=1,sd=0.2,rf=0.03,iter=2**15)
Prix_CallBS = CallBS(10,K=10,T=1,sd=0.2,rf=0.03)
#print(f'Prix du call\nMonte Carlo {Prix_CallMC}\nBlack And Scholes {Prix_CallBS}')

Prix du call:

Monte Carlo 0.9418881886630102

Black And Scholes 0.9413403383853005

En développant passant au $$\ln(.)$$ L'expression de $$S_T = S_0 * exp\Big((r-\frac{\sigma^2}{2} )*T+\sigma\sqrt{T}*\epsilon \Big)$$ On peut connaitre la loi suivie par $$S_T$$ Ensuite on détermine un intervalle de confiance de $$ln(S_T)$$ Selon le seuil $$\alpha$$ Puis on passe chaque borne à l'exponentielle afin d'avoir un intervalle de confinance de $$S_T$$


In [106]:
Call_OTM_MC = CallMonteCarlo(10,15,1,0.2,0.03,2**15)
Call_OTM_BS = CallBS(10,15,1,0.2,0.03)
#print(f'Prix du call Out of the money\nMonte Carlo {Call_OTM_MC}\nBlack And Scholes {Call_OTM_BS}')

Prix du call Out of the money

Monte Carlo 0.029731738074301125

Black And Scholes 0.028177817015465567

In [107]:
def CallMonteCarlo_Antithetic(S0,K,T,sd,rf,iter):
    C = np.mean([([Call(rf,T,AssetValueByMC(S0,T,sd,rf,(i)),K),
                          Call(rf,T,AssetValueByMC(S0,T,sd,rf,-(i)),K)
                            ]) for i in np.random.normal(0,1,iter)])
    return C

In [108]:
C = CallMonteCarlo_Antithetic(10,10,1,0.2,0.03,2**15)
C

0.9437542135945908

Avec une variable antithétique on a des résultats qui se rapprochent légèrement du prix B&S avec une variance inférieure au cas sans variable antithétique

Pour la suite on peut appliquer les mêmes résultats pour avoir un put, puis utiliser la relation de parité call-put afin d'avoir méthode de réduction de la variance nommée control variates en sachant que la relation de partié nous donne un estimateur sans biais du prix d'un call sachant les autres paramètres de la formule. $$Call^* = Call_{{MC}} + c * (Put_{MC}-Put_{Parité})$$

In [109]:
def Put(rf,T,ST,K):
    C = np.exp(-rf*T)*np.maximum(0,K-ST)
    return C

def PutMonteCarlo(S0,K,T,sd,rf,iter):
    C = np.mean([Put(rf,T,AssetValueByMC(S0,T,sd,rf,i),K) for i in np.random.normal(0,1,iter)])
    return C

def Call_MC_ControlVariate(S0,K,T,sd,rf,iter):
    S_T = [AssetValueByMC(S0,T,sd,rf,i) for i in np.random.normal(0,1,iter)]
    Call_MC = [Call(rf,T,i,K) for i in S_T]
    MatriceCOV = np.cov(Call_MC,S_T)
    Call_MC = np.mean(Call_MC)
    C_opt = MatriceCOV[0][1]/MatriceCOV[1][1]
    Put_parite = Call_MC - S0 + K * np.exp(-rf*T)
    Put_MC = np.mean([Put(rf,T,i,K) for i in S_T])
    return Call_MC+C_opt*(Put_MC-Put_parite)


In [110]:
Call_MC_ControlVariate(10,10,1,0.2,0.03,2**18)

0.9409473848001869

La variable de contrôle permet d'affiner les résultats en réduisant la variance